In [1]:
# Import CSV into pandas df
# clean file name and remove all extra characters which can be compatible with table name
# clean files headers which can be compatible with table columns name
# create table
# insert data into table

In [63]:
import pandas as pd 
import psycopg2 as pg

In [64]:
# read data from csv ans store in pandas df
!ls 

001-ImportCSVtoDfToDatabase.ipynb     002-ImportCSVToDatabaseAutomate.ipynb


In [65]:
!ls ../data/

Customer Contracts$.csv   Customer Engagements.csv  survey_results_public.csv
Customer Demo.csv         csv_import_functions.py   survey_results_schema.csv


In [66]:
customer_contract_df = pd.read_csv('../data/Customer Contracts$.csv')

In [67]:
customer_contract_df.head()

,customer_name,start_date,end_date,contract_amount_m,invoice_sent,paid
0,Nike,01-02-2019,12-20-2020,2.98,Yes,Yes
1,Reebox,06-20-2017,NaN,3.90,No,No
2,Adidas,12-07-2015,6-20-2018,4.82,Yes,Yes
3,Google,05-25-2014,03-20-2017,5.74,Yes,No
4,Amazon,11-10-2012,12-20-2015,6.66,No,Yes


In [68]:
# To get the counts of row and column in df/csv
customer_contract_df.shape

(10, 6)

In [69]:
# Since we have file name with space and $ or there may be possibility we have unwanted characters
# So lets remove unwanted spaces
# remove unwanted characters
# make lowercase
# replace space with underscore
file = 'Customer Contracts$'
parsed_file = file.lower().replace(' ', '_').replace('?', '').replace('$','').replace('-','_') \
                .replace('\\', '_').replace(r'/','_').replace(r',','_')

In [70]:
parsed_file

'customer_contracts'

In [71]:
# lets rename all the column names of df
customer_contract_df.columns

Index(['customer_name', 'start_date', 'end_date', 'contract_amount_m',
       'invoice_sent', 'paid'],
      dtype='object')

In [72]:
customer_contract_df.columns = [x.lower().replace(' ', '_').replace('?', '').replace('$','').replace('-','_') \
                .replace('\\', '_').replace(r'/','_').replace(r',','_') for x in customer_contract_df.columns]

In [73]:
customer_contract_df.columns

Index(['customer_name', 'start_date', 'end_date', 'contract_amount_m',
       'invoice_sent', 'paid'],
      dtype='object')

In [74]:
customer_contract_df.dtypes

customer_name         object
start_date            object
end_date              object
contract_amount_m    float64
invoice_sent          object
paid                  object
dtype: object

In [75]:
replacements = {
    'object': 'VARCHAR',
    'float64': 'FLOAT',
    'int64': 'INTEGER',
    'datetime64': 'TIMESTAMP',
    'timedelta64[ns]': 'VARCHAR'
}

In [76]:
# CREATE TABLE customer_contract (
#     customer_name VARCHAR(255),
#     start_date VARCHAR,
#     end_date VARCHAR,
#     contract_amount_m FLOAT,
#     invoice_sent VARCHAR,
#     paid VARCHAR
# )

In [77]:
# Lets automate create table statement
col_str = ', '.join('{} {}'.format(col_name, data_type) for (col_name, data_type) in zip(customer_contract_df.columns, customer_contract_df.dtypes.replace(replacements)))

In [78]:
col_str

'customer_name VARCHAR, start_date VARCHAR, end_date VARCHAR, contract_amount_m FLOAT, invoice_sent VARCHAR, paid VARCHAR'

In [79]:
def connect_to_db():
    try:
        conn = pg.connect(host=hostname, port=port, user=username, password=password)
    except ConnectionError as e:
        raise e
    else:
        print('Connected successfully....')
    return conn

In [80]:
# Lets connect to database
hostname='localhost'
port='5433'
username='customeruser'
password='superuser'
database='customer'
conn=None

In [81]:
conn = connect_to_db()
curr = conn.cursor()

Connected successfully....


In [82]:
# DROP table if there are any existing table --- BE CAREFUL, may be its not needed in your case
def drop_table(curr, table_name):
    query = """DROP TABLE IF EXISTS {};"""
    curr.execute(query.format(table_name))

In [83]:
# curr.execute('ROLLBACK')
drop_table(curr, parsed_file)

In [84]:
def create_table(curr, table_name, cols):
    query = """CREATE TABLE IF NOT EXISTS {} ( {} );"""
    curr.execute(query.format(table_name, cols))
    

In [85]:
create_table(curr, parsed_file, col_str)
conn.commit()

In [87]:
# Save df data to csv file
customer_contract_df.to_csv('./customer_contracts.csv', header=customer_contract_df.columns, index=False, encoding='UTF-8')
print('Saved df data to csv ....')
#Open csv file , save it as an object and upload to db
my_file = open('./customer_contracts.csv')
print('Opened file in memory...')

Saved df data to csv ....
Opened file in memory...


In [88]:
# NOW IMPORT CSV DATA TO DATABASE
# Upload csv to db
SQL_STATEMENT = """
COPY customer_contracts FROM STDIN WITH CSV HEADER DELIMITER AS ','
"""


In [91]:
curr.copy_expert(sql=SQL_STATEMENT, file=my_file)
conn.commit()

In [92]:
curr.close()
print('Table get populated with csv file data completed..')

Table get populated with csv file data completed..
